<a href="https://colab.research.google.com/github/park-geun-hyeong/practice_pytorch/blob/main/Rogistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


## 직접 sigmoid, cross_entropy함수를 구현함으로써 model 생성

In [69]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,2],[6,4]]
y_data = [[0],[0],[0],[1],[1],[1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data) 

W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = 1/ (1+ torch.exp(-(x_train.matmul(W)+b))) ## Sigmoid(Wx+b) 형태
losses = -(y_train*torch.log(hypothesis) + (1-y_train)*torch.log(1-hypothesis))   ## Cross_Entropy Loss
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [59]:
optimizer = optim.SGD([W,b],lr=0.01)


for epoch in tqdm(range(10000)):
    hypothesis = 1 / (1+torch.exp(-(x_train.matmul(W)+b)))
    losses = -(y_train*torch.log(hypothesis) + (1-y_train)*torch.log(1-hypothesis))
    cost = losses.mean() 

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 1000==0:
        print(f'epoch:{epoch}, loss: {cost.item()}')

  6%|▌         | 592/10000 [00:00<00:03, 2883.66it/s]

epoch:0, loss: 0.6931471824645996


 14%|█▍        | 1444/10000 [00:00<00:02, 2856.32it/s]

epoch:1000, loss: 0.41277217864990234


 23%|██▎       | 2293/10000 [00:00<00:02, 2831.78it/s]

...

 84%|████████▎ | 8353/10000 [00:02<00:00, 2828.43it/s]

epoch:8000, loss: 0.17314155399799347


 95%|█████████▌| 9533/10000 [00:03<00:00, 2912.04it/s]

epoch:9000, loss: 0.1602088063955307


100%|██████████| 10000/10000 [00:03<00:00, 2826.51it/s]


In [60]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)

In [61]:
hypothesis

tensor([[0.0277],
        [0.1463],
        [0.3141],
        [0.7827],
        [0.9265],
        [0.9900]], grad_fn=<SigmoidBackward>)

In [62]:
prediction = hypothesis>=torch.Tensor([0.5])
prediction

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

## nn.Sequentail 함수를 이용한 model 생성

In [63]:
model = nn.Sequential(
    nn.Linear(2,1),
    nn.Sigmoid()
)

In [76]:
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [77]:
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    logits = model(x_train)
    loss = F.binary_cross_entropy(logits, y_train)

    loss.backward()
    optimizer.step()

    if epoch % 100==0:
        print(f'epoch:{epoch}, loss: {loss.item()}')

 52%|█████▏    | 523/1000 [00:00<00:00, 2571.07it/s]

epoch:0, loss: 0.07493040710687637
epoch:100, loss: 0.06048277020454407
epoch:200, loss: 0.04977576807141304
epoch:300, loss: 0.041668329387903214
epoch:400, loss: 0.03538207337260246
epoch:500, loss: 0.030408544465899467


100%|██████████| 1000/1000 [00:00<00:00, 2682.11it/s]

epoch:600, loss: 0.02640472911298275
epoch:700, loss: 0.02313263900578022
epoch:800, loss: 0.0204231608659029
epoch:900, loss: 0.018153365701436996


In [80]:
print(list(model.parameters()))

[Parameter containing:
tensor([[3.5560, 1.5782]], requires_grad=True), Parameter containing:
tensor([-15.5635], requires_grad=True)]


In [82]:
model(x_train)

tensor([[1.4321e-04],
        [2.3734e-02],
        [3.4983e-02],
        [9.6755e-01],
        [9.9538e-01],
        [9.9999e-01]], grad_fn=<SigmoidBackward>)

## Class를 이용한 Model 생성

In [92]:
 class BinaryClassifier(nn.Module):
     def __init__(self):
        super().__init__()      

        self.linear = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()

     def forward(self, x):
         ret = self.sigmoid(self.linear(x))
         return ret

In [93]:
model2 = BinaryClassifier()

In [95]:
optimizer = optim.Adam(model2.parameters(),lr=0.01)

In [99]:
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    logits = model2(x_train)
    loss = F.binary_cross_entropy(logits, y_train)

    loss.backward()
    optimizer.step()

    if epoch % 100==0:
        print(f'epoch:{epoch}, loss: {loss.item()}')

 57%|█████▋    | 567/1000 [00:00<00:00, 2758.47it/s]

epoch:0, loss: 0.026065044105052948
epoch:100, loss: 0.024472856894135475
epoch:200, loss: 0.02299751155078411
epoch:300, loss: 0.021628396585583687
epoch:400, loss: 0.020356101915240288
epoch:500, loss: 0.01917225308716297


100%|██████████| 1000/1000 [00:00<00:00, 2816.20it/s]

epoch:600, loss: 0.018069244921207428
epoch:700, loss: 0.017040394246578217
epoch:800, loss: 0.016079626977443695
epoch:900, loss: 0.015181489288806915


In [102]:
hypothesis2 = model2(x_train)

In [103]:
prediction = hypothesis2 >= torch.Tensor([0.5])
prediction

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

In [104]:
torch.mean((y_train==prediction).float())

tensor(1.)